In [30]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
plt.rcParams.update({'font.size': 14})
import matplotlib.gridspec as gridspec
import datetime
%config InlineBackend.figure_format = 'retina'
from datetime import datetime, date

In [33]:
# import regression parameters
regression_params = pd.read_csv('../Data/regression_temperature_performance.csv')
for param in ['summer_params','winter_params','spring_params','fall_params']:
    regression_params[param] = [p[1:-1].split() for p in regression_params[param].values]
    regression_params[param] = [[float(p[i][:-1]) for i in range(len(p))] for 
                                    p in regression_params[param].values]
regression_params['sitenum'] = ['0'*(8-len(str(s)))+str(s) for s in regression_params.sitenum.values]

# import temperature data
gridmet_tmin = pd.read_csv('../Data/gridmet_tempsites/gridmet_Tmin_C_15yr_temp_gages.csv')
gridmet_tmin['date'] = pd.to_datetime(gridmet_tmin['date'])
gridmet_tmin = gridmet_tmin.set_index('date')

gridmet_tmax = pd.read_csv('../Data/gridmet_tempsites/gridmet_Tmax_C_15yr_temp_gages.csv')
gridmet_tmax['date'] = pd.to_datetime(gridmet_tmax['date'])
gridmet_tmax = gridmet_tmax.set_index('date')
gridmet_tmin['DOY'] = [date(gridmet_tmin.index.year.values[i], 
                            gridmet_tmin.index.month.values[i], 
                            gridmet_tmin.index.day.values[i]).timetuple().tm_yday
                      for i in range(len(gridmet_tmin))]
gridmet_tmax['DOY'] = gridmet_tmin['DOY']

In [64]:
def streamT(X, a, b, c):
    airT, Q = X
    return a*airT + b*Q + c

In [124]:
num_iterations = 100
num_years = 50
winter_days = [1,92]
spring_days = [92,183]
summer_days = [183,275]
fall_days = [275,366]

site = '01095220'
qdata = pd.read_csv('../Data/DepletedQ_Dry_Seasonal_'+str(site)+'.csv')
qdata = qdata.set_index('Day')
siteparam = regression_params[regression_params.sitenum.astype('str')==site]

keys = ['DOY','airT','T_C']
keys1 = ['T_C_dplt'+str(i) for i in range(1,num_iterations+1)]
keys = keys+keys1

tdata = {key:[] for key in keys}
tdata['DOY'] = qdata.DOY.values

# get median air temperature
site_tmin = gridmet_tmin[[site,'DOY']].groupby('DOY').median()
site_tmax = gridmet_tmax[[site,'DOY']].groupby('DOY').median()
site_tair = ((site_tmin+site_tmax)/2)
tdata['airT'] = (list(site_tair.values.flatten()))*num_years

# formula for temperature model requires streamflow in CFS
qdata_cfs = qdata
for col in qdata_cfs.columns:
    if col != 'DOY':
        qdata_cfs[col] = qdata_cfs[col]*100**3/2.54**3/12**3

# apply formula for temperature model to each streamflow timeseries
for col in qdata_cfs.columns:
    if col !='DOY':
        param = siteparam.winter_params.values[0]
        tseries = np.where((qdata.DOY>=winter_days[0]) & (qdata.DOY<winter_days[1]),
                           [streamT((tdata['airT'][i],qdata[col].values[i]),*param)
                           for i in range(len(qdata))],
                           np.nan)
        param = siteparam.spring_params.values[0]
        tseries = np.where((qdata.DOY>=spring_days[0]) & (qdata.DOY<spring_days[1]),
                           [streamT((tdata['airT'][i],qdata[col].values[i]),*param)
                           for i in range(len(qdata))],
                           tseries)
        param = siteparam.summer_params.values[0]
        tseries = np.where((qdata.DOY>=summer_days[0]) & (qdata.DOY<summer_days[1]),
                           [streamT((tdata['airT'][i],qdata[col].values[i]),*param)
                           for i in range(len(qdata))],
                           tseries)
        param = siteparam.fall_params.values[0]
        tseries = np.where((qdata.DOY>=fall_days[0]) & (qdata.DOY<fall_days[1]),
                           [streamT((tdata['airT'][i],qdata[col].values[i]),*param)
                           for i in range(len(qdata))],
                           tseries)
        if col == 'Q_cms':
            tdata['T_C'] = tseries
        else:
            end = col.split('_')[-1]
            tdata['T_C_'+end] = tseries
            
tdata = pd.DataFrame.from_dict(tdata)
tdata.to_csv('../Data/depleted_temperature_timeseries/depleted_temp_'+site+'.csv',index=False)

In [ ]:
# tdata.T_C.plot()
for c in tdata.columns:
    if c not in ['DOY','airT']:
        tdata[c].plot(label = c)
plt.xlim(0,600)
plt.legend()
# plt.ylim(0,25)